In [27]:
import numpy as np
import statsmodels.api as sm
from ISLP import load_data
from ISLP.models import (ModelSpec as MS, summarize, poly)
from sklearn.model_selection import train_test_split

In [28]:
from functools import partial
from sklearn.model_selection import \
(cross_validate, KFold, ShuffleSplit)
from sklearn.base import clone
from ISLP.models import sklearn_sm

In [29]:
#Validation Set Approach

Auto = load_data('Auto') 
Auto_train, Auto_valid = train_test_split(Auto, test_size = 196, random_state = 0)

In [30]:
hp_mm = MS(['horsepower'])
X_train = hp_mm.fit_transform(Auto_train)
y_train = Auto_train['mpg']
model = sm.OLS(y_train, X_train)
results = model.fit()

In [31]:
X_valid = hp_mm.transform(Auto_valid)
y_valid = Auto_valid['mpg']
valid_pred = results.predict(X_valid) 
np.mean((y_valid-valid_pred)**2) #calculate validation MSE of the linear regression

23.61661706966988

In [32]:
def evalMSE(terms, response, train, test): 
    
    mm = MS(terms)
    X_train = mm.fit_transform(train)#training set
    y_train = train[response]
    X_test = mm.transform(test)#test set
    y_test = test[response]
    results = sm.OLS(y_train, X_train).fit()
    test_pred = results.predict(X_test)
    return np.mean((y_test-test_pred)**2)
#evalMSE를 사용하여 training set과 test set의 MSE도출


In [33]:
MSE = np.zeros(3)
for idx, degree in enumerate(range(1,4)):
    MSE[idx] = evalMSE([poly('horsepower', degree)],
                      'mpg',
                      Auto_train, Auto_valid)
print(MSE) 

[23.61661707 18.76303135 18.79694163]


In [34]:
Auto_train, Auto_valid = train_test_split(Auto, test_size = 196, random_state = 3)
MSE = np.zeros(3)
for idx, degree in enumerate(range(1,4)):
    MSE[idx] = evalMSE([poly('horsepower', degree)], 'mpg', Auto_train, Auto_valid)
print(MSE) #우리는 이를 통해 각각 일차, 이차 삼차 모델의 검증셋 오차율을 살펴볼 수 있다.
#성능: 이차모델>삼차모델>일차모델

[20.75540796 16.94510676 16.97437833]


In [35]:
#Cross Validation
hp_model = sklearn_sm(sm.OLS, MS(['horsepower']))
X,Y = Auto.drop(columns = ['mpg']), Auto['mpg']
cv_results = cross_validate(hp_model, X, Y, cv = Auto.shape[0])
cv_err = np.mean(cv_results['test_score'])
cv_err#cross validated test score

24.231513517929216

In [36]:
cv_error = np.zeros(5)
H = np.array(Auto['horsepower'])
M = sklearn_sm(sm.OLS)
for i, d in enumerate(range(1,6)):
    X = np.power.outer(H, np.arange(d+1))
    M_CV = cross_validate(M, X, Y, cv = Auto.shape[0])
    cv_error[i] = np.mean(M_CV['test_score'])
print(cv_error)

[24.23151352 19.24821312 19.33498406 19.42443031 19.03320903]


In [38]:
A = np.array([3, 5, 9])
B = np.array([2, 4])
np.add.outer(A,B) #A와 B의 합산식에 대한 array생성

array([[ 5,  7],
       [ 7,  9],
       [11, 13]])

In [39]:
cv_error = np.zeros(5)
cv = KFold(n_splits=10, shuffle=True, random_state=0)
for i, d in enumerate(range(1,6)):
    X = np.power.outer(H, np.arange(d+1))
    M_CV = cross_validate(M,X,Y, cv=cv) #cv = Auto.shpae[0] vs cv = KFold()
    cv_error[i] = np.mean(M_CV['test_score'])
cv_error

array([24.20766449, 19.18533142, 19.27626666, 19.47848402, 19.13722633])

In [43]:
validation = ShuffleSplit(n_splits=1, test_size=196, random_state=0)
results = cross_validate(hp_model, 
                         Auto.drop(['mpg'], axis=1),
                         Auto['mpg'], cv = validation);
results['test_score']

array([23.61661707])

In [44]:
validation = ShuffleSplit(n_splits=10, test_size=196, random_state=0)
#Shufflesplit 함수는 데이터셋 인덱스를 무작위로 사전에 설정한 비율로 분할한다.
results = cross_validate(hp_model, 
                        Auto.drop(['mpg'], axis=1),
                         Auto['mpg'], cv = validation)
results['test_score'].mean(), results['test_score'].std()

(23.802232661034164, 1.4218450941091847)

In [49]:
# The Bootstrap

Portfolio = load_data('Portfolio')
def alpha_func(D, idx):
    cov_ = np.cov(D[['X','Y']].loc[idx], rowvar=False)
    return ((cov_[1,1] - cov_[0,1])/(cov_[0,0]+cov_[1,1]-2*cov_[0,1]))
#alpha_func 함수는 관측된 idx에 대해 추정치 alpha를 최소제곱법에 근거하여 도출한다. 
alpha_func(Portfolio, range(100))
#100개의 관측값에 대해 alpha값을 도출

0.57583207459283

In [50]:
rng = np.random.default_rng(0)
alpha_func(Portfolio, rng.choice(100,100, replace=True))

0.6074452469619004

In [67]:
def boot_SE(func, D, n=None, B=1000, seed=0):
    rng = np.random.default_rng(seed)
    first_, second_ = 0, 0
    n = n or D.shape[0]
    for _ in range(B):
        idx = rng.choice(D.index, n, replace=True)
        value = func(D, idx)
        first_ += value
        second_ += value**2
    return np.sqrt(second_ / B - (first_ / B)**2)
#B만큼의 루프가 반복된다.

In [68]:
alpha_SE = boot_SE(alpha_func, Portfolio, B=1000, seed=0)
alpha_SE
# 추정치 alpha hat의 최종 standard error

0.09118176521277699

In [69]:
# Estimating the Acuuracy of a Linear Regression Model
def boot_OLS(model_matrix, response, D, idx):
    D_ = D.loc[idx]
    Y_ = D_[response]
    X_ = clone(model_matrix).fit_transform(D_)
    return sm.OLS(Y_, X_).fit().params
# clone function: copy formula

In [70]:
hp_func = partial(boot_OLS, MS(['horsepower']), 'mpg')
# partial function: parameter의 값이 미리 채워진 상태로 반환된다. 
rng = np.random.default_rng(0)
np.array([hp_func(Auto, rng.choice(392, 392, replace=True)) for _ in range(10)])
# 10개의 붓스트랩 샘플 생성

array([[39.88064456, -0.1567849 ],
       [38.73298691, -0.14699495],
       [38.31734657, -0.14442683],
       [39.91446826, -0.15782234],
       [39.43349349, -0.15072702],
       [40.36629857, -0.15912217],
       [39.62334517, -0.15449117],
       [39.0580588 , -0.14952908],
       [38.66688437, -0.14521037],
       [39.64280792, -0.15555698]])

In [73]:
hp_se = boot_SE(hp_func, Auto, B=1000, seed=10)
hp_se
# 결괏값을 보면 beta zero hat 의 SE는 0.85, beta one hat의 SE는 0.0074이다.

intercept     0.848807
horsepower    0.007352
dtype: float64

In [74]:
hp_model.fit(Auto, Auto['mpg'])
model_se = summarize(hp_model.results_)['std err']
model_se

intercept     0.717
horsepower    0.006
Name: std err, dtype: float64

In [75]:
quad_model = MS([poly('horsepower', 2, raw=True)])
quad_func = partial(boot_OLS, quad_model, 'mpg')
boot_SE(quad_func, Auto, B=1000)
# 선형모델이 아니라 이차모델을 사용한다. 

intercept                                  2.067840
poly(horsepower, degree=2, raw=True)[0]    0.033019
poly(horsepower, degree=2, raw=True)[1]    0.000120
dtype: float64

In [76]:
M = sm.OLS(Auto['mpg'], quad_model.fit_transform(Auto))
summarize(M.fit())['std err']

intercept                                  1.800
poly(horsepower, degree=2, raw=True)[0]    0.031
poly(horsepower, degree=2, raw=True)[1]    0.000
Name: std err, dtype: float64